In [111]:
from datetime import datetime, date, time, timedelta
from itertools import islice
from geopy.distance import great_circle
from operator import itemgetter
import matplotlib.pylab as plt
from pandas import DataFrame
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy
import json
import math


DELTA_GNSS = 0.05 #equivalent to about 5-6 km
DELTA_GNSS_INIT = 1 #equivalent to about 5-6 km
DELTA_POS = 0.0001 #equivalent to about 100 m

#six decimal are enough for a decimeter precision
#0.1 = 11.1 km
#0.01 = 1.11 km
#...
#0.000001 = 0.111 m
gw_latitude = 45.004831
gw_longitude = 10.467967
gw_coo = (gw_latitude, gw_longitude)

#support variables
bts_latitude = 0
bts_longitude = 0

#Import UAV Log network Ping data from JSON file
data = []
with open('LORAWAN_TTGO.json') as f:
    for line in f:
        try:
            data.append(json.loads(line))
        except:
            print('Skip line')

#Convert JSON data from Python dict to DF
UAV_LOG = pd.json_normalize(data)

#Drop unusfull data
#UAV_LOG = UAV_LOG.drop(columns=['uav_param.status','uav_param.mode','interface','gps_param.groundspeed','gps_param.velocity.vx','uav_param.attitude.pitch','uav_param.attitude.yaw','uav_param.attitude.roll','gps_param.velocity.vy','gps_param.velocity.vz','gps_param.hdop','gps_param.vdop','gps_param.fix_type'])

#Remove duplicate entries
UAV_LOG = UAV_LOG.drop_duplicates('timestamp')
UAV_LOG = UAV_LOG.apply(pd.to_numeric, errors='ignore')

#Remove false flights
#UAV_bench = UAV_LOG['uav_param.battery']>=10.0
#UAV_LOG = UAV_LOG[UAV_bench]
l = UAV_LOG['gateways']

#l[1]['channel_rssi']
#print(d)
#print(l['gateway_ids']['gateway_id'])

for i in range(len(l)):
    d = pd.json_normalize(l[i])
    print(str(d['gateway_ids.gateway_id']))
    #if (d['gateway_ids.gateway_id'] == 'uniprgw'):
    #    print(d['channel_rssi'])
#Remove no GPS fix data
#GPS_Null = UAV_LOG['gps_param.satellites']>=6
#UAV_LOG = UAV_LOG[GPS_Null]

#Sort by time for time clustering
#UAV_LOG = UAV_LOG.sort_values(by='timestamp')

0    uniprgw
Name: gateway_ids.gateway_id, dtype: object
0    uniprgw
Name: gateway_ids.gateway_id, dtype: object
0    uniprgw
Name: gateway_ids.gateway_id, dtype: object
0    uniprgw
Name: gateway_ids.gateway_id, dtype: object
0    uniprgw
Name: gateway_ids.gateway_id, dtype: object
0    uniprgw
Name: gateway_ids.gateway_id, dtype: object
0    uniprgw
Name: gateway_ids.gateway_id, dtype: object
0    uniprgw
Name: gateway_ids.gateway_id, dtype: object
0    uniprgw
Name: gateway_ids.gateway_id, dtype: object
0    uniprgw
Name: gateway_ids.gateway_id, dtype: object
0    uniprgw
Name: gateway_ids.gateway_id, dtype: object
0    uniprgw
Name: gateway_ids.gateway_id, dtype: object
0    uniprgw
Name: gateway_ids.gateway_id, dtype: object
0    uniprgw
Name: gateway_ids.gateway_id, dtype: object
0    uniprgw
Name: gateway_ids.gateway_id, dtype: object
0    uniprgw
Name: gateway_ids.gateway_id, dtype: object
0    uniprgw
Name: gateway_ids.gateway_id, dtype: object
0    uniprgw
Name: gateway_ids.

In [112]:
from ipyleaflet import Map, basemaps, Marker, Icon
from ipywidgets import Layout

zoom = 25

icon_drone = Icon(icon_url='https://raw.githubusercontent.com/palia95/lora-Localization/main/icons/drone.png', icon_anchor=[16, 16], icon_size=[44, 44], rotate=True)
icon_bts = Icon(icon_url='http://simpleicon.com/wp-content/uploads/antenna-2-64x64.png', icon_anchor=[32, 32], icon_size=[32, 32])

#Remove log for same points
UAV_LOG = UAV_LOG.round(6)
UAV_LOG = UAV_LOG.drop_duplicates('latitude')
UAV_LOG = UAV_LOG.drop_duplicates('longitude')

m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=gw_coo, zoom=zoom, layout=Layout(width='100%', height='800px'))

def MarkerDrone(uav_coo):
    m.add_layer(Marker(location=uav_coo, icon=icon_drone, draggable=False))

def MarkerGW(gw_coo):
    m.add_layer(Marker(location=gw_coo, icon=icon_bts, draggable=False))

    
uav_flight = []
log = 0

MarkerGW(gw_coo)

for log in range(len(UAV_LOG)):
    uav_coo = (UAV_LOG.iloc[log]['latitude'], UAV_LOG.iloc[log]['longitude'])
    distance = great_circle(uav_coo, gw_coo).m
    l = UAV_LOG.iloc[log]['gateways']

    for i in range(len(l)):
        if (l[i]['gateway_ids']['gateway_id'] == 'uniprgw'):
            rssi = l[i]['channel_rssi']
            snr = l[i]['snr']

            uav_flight.append((UAV_LOG.iloc[log]['timestamp'], UAV_LOG.iloc[log]['latitude'], UAV_LOG.iloc[log]['longitude'], UAV_LOG.iloc[log]['altitude'], distance, rssi, snr))
            MarkerDrone(uav_coo)
    #print(BTS.iloc[0])
    #print(DF.iloc[log])
#uav_flight

m

Map(center=[45.004831, 10.467967], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

In [115]:
DF = DataFrame (uav_flight)

In [118]:
DF.columns = ["time", "latitude", "longitude", "altitude", "distance", "rssi", "snr"]
DF

,time,latitude,longitude,altitude,distance,rssi,snr
0,2022-07-19T07:03:47.036096458Z,45.004825,10.467922,16,3.600265,-31,13.0
1,2022-07-19T07:05:07.434762820Z,45.004836,10.467836,14,10.314237,-37,13.5
2,2022-07-19T07:05:27.478161442Z,45.004868,10.467901,13,6.622068,-43,10.5
3,2022-07-19T07:10:28.716715410Z,45.005136,10.468008,22,34.067342,-57,14.0
4,2022-07-19T07:10:48.804194109Z,45.005426,10.468158,21,67.843786,-59,13.8
5,2022-07-19T07:11:08.853045904Z,45.005887,10.468180,26,118.610095,-75,11.8
6,2022-07-19T07:11:28.934541344Z,45.006617,10.468223,31,199.611667,-74,13.0
7,2022-07-19T07:11:49.015840975Z,45.007658,10.468201,31,314.886357,-68,14.0
8,2022-07-19T07:12:09.112394359Z,45.008666,10.468051,30,426.484278,-70,11.0
9,2022-07-19T07:12:29.172815746Z,45.009632,10.467815,30,533.981323,-74,13.8


In [28]:
from ipyleaflet import Map, basemaps, Marker, Icon
from ipywidgets import Layout

zoom = 25
DF = UAV_LOG

icon_drone = Icon(icon_url='https://raw.githubusercontent.com/palia95/LoRa-Localization/main/icons/drone_plane.png', icon_anchor=[16, 16], icon_size=[44, 44], rotate=True)
icon_bts = Icon(icon_url='http://simpleicon.com/wp-content/uploads/antenna-2-64x64.png', icon_anchor=[32, 32], icon_size=[32, 32])

#Remove log for same points
#DF = DF.round(5)
DF = DF.drop_duplicates('latitude')
DF = DF.drop_duplicates('longitude')

m = Map(basemap=basemaps.OpenStreetMap.Mapnik, zoom=zoom, layout=Layout(width='100%', height='800px'))

def MarkerDrone(uav_coo):
    m.add_layer(Marker(location=uav_coo, icon=icon_drone, draggable=False))

def MarkerGW(gw_coo):
    m.add_layer(Marker(location=gw_coo, icon=icon_bts, draggable=False))




m

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…